In [2]:
#1. Написать бота который будет проверять доступность сайта
import telebot
import requests
bot = telebot.TeleBot('токен')
@bot.message_handler(content_types=['text'])
def start_message(message):
    if message.text == '/start':
        bot.send_message(message.chat.id, 'Я на связи. Напиши мне сайт, который ты хочешь проверить')
        bot.register_next_step_handler(message, get_url)
    else:
        bot.send_message(message.chat.id, 'Напиши /start')

def get_url(message):
    global url
    url = message.text
    bot.send_message(message.chat.id, 'Подожди, сейчас проверю')  
    try:
        webpage = requests.get(url)
        if webpage.status_code == 200:
            bot.send_message(message.chat.id, 'Сайт доступен!')
        else:
            bot.send_message(message.chat.id, 'Сайт сдох!')
    except:
        bot.send_message(message.chat.id, 'Что-то не то')

bot.polling(none_stop=True, interval=0)

In [1]:
# 2. Написать бота который будет анализировать текст - (Статистика, количество уникальных слов, самые популярные слова, кроме союзов и предлогов, количество предложений)
import telebot
import re
import time
bot = telebot.TeleBot('токен')
@bot.message_handler(content_types=['text'])
def start_message(message):
    if message.text == '/start':
        bot.send_message(message.chat.id, 'Напиши мне текст и я его проанализирую')
        bot.register_next_step_handler(message, analytics)
    else:
        bot.send_message(message.chat.id, 'Напиши /start')

def analytics(message):
    global txt
    txt = message.text
#     print(message)
    splitted = re.split(r'[-;,!?.—\n\t_ ]', txt.lower())
    spisok_vse = [i for i in splitted if not i == '']
    list_ex = ['на', 'и', 'но', 'да', 'если', 'как', 'тоже', 'хотя', 'чтобы', 'с', 'то', 'за', 'про', 'ли', 'так', 'в', 'к', 'а', 'о', 'по', 'под', 'подо', 'ведь', 'ото', 'для']
    global spisok
    spisok = [word for word in spisok_vse if word not in list_ex]
    bot.send_message(message.chat.id, text = 'Предложений в тексте: ' + str(sentenses()))
    time.sleep(0.25)
    bot.send_message(message.chat.id, text = longest_word())
    time.sleep(0.25)
    bot.send_message(message.chat.id, text = unique_words())
    time.sleep(0.25)
    bot.send_message(message.chat.id, text = popular_word())
    
def sentenses():
    amount = txt.count('.') + txt.count('!') + txt.count('?')
    if amount == 0:
        amount = 1
    return amount

def longest_word():
    long_len = 0
    longest = ''
    for i in spisok:
        if len(i) > long_len:
            long_len = len(i)
            longest = i
    hah = f'Самое длинное слово: {longest}\nЕго длина в символах: {long_len}'
    return hah

def popular_word():
    try:
        count = 0
        word = spisok[0]
        for i in spisok:
            if spisok.count(i) > count:
                count = spisok.count(i)
                word = i
        if count > 1:
            heh = f'Самое популярное слово в тексте: {word} \nОно встречается {count} раз(а)'
        else:
            heh = 'Все слова в тексте уникальные'
    except:
        heh = 'У тебя тут нет слов, дурачок!'  
    return heh

def unique_words():
    uni = 0
    uni_word = ''
    for i in spisok:
        if spisok.count(i) == 1:
            uni_word += i + ", "
            uni += 1
    uni_word = uni_word[0:-2]
    uni_stat = f'Уникальные слова в тексте: {uni_word} \nЭтих слов аж {uni}'
    return uni_stat
    
bot.polling(none_stop=True, interval=0)

In [13]:
#3 Бот калькулятор
import telebot
import time
bot = telebot.TeleBot('токен')
@bot.message_handler(content_types=['text'])
def start_message(message):
    if message.text == '/start':
        bot.send_message(message.chat.id, 'Напиши мне простое выражение и я выдам тебе результат')
        time.sleep(0.5)
        bot.send_message(message.chat.id, 'Напиши первое число')
        bot.register_next_step_handler(message, get_first)
    else:
        bot.send_message(message.chat.id, 'Напиши /start')
        
def get_first(message):
    global first
    first = message.text
    try:
        first = float(first)
        bot.send_message(message.chat.id, 'Теперь введи второе число')
        bot.register_next_step_handler(message, get_second)
    except:
        bot.send_message(message.chat.id, 'Ты ввел не число!\nТеперь все сначала вводи!')
        
def get_second(message):
    global second
    second = message.text
    try:
        second = float(second)
        bot.send_message(message.chat.id, 'Теперь введи операцию (+, -, *, /, ^)')
        bot.register_next_step_handler(message, get_operand)
    except:
        bot.send_message(message.chat.id, 'Ты ввел не число!\nТеперь все сначала вводи!')
        
def get_operand(message):
    global operand
    operand = message.text
    try:
        if operand == '+' or operand == '-' or operand == '*' or operand == '/' or operand == '^':
            res = 0
            if operand == '+':
                res = first + second
                bot.send_message(message.chat.id, text = str(res))
            elif operand == '-':
                res = first - second
                bot.send_message(message.chat.id, text = str(res))
            elif operand == '*':
                res = first * second
                bot.send_message(message.chat.id, text = str(res))
            elif operand == '^':
                res = first ** second
                bot.send_message(message.chat.id, text = str(res))
            elif operand == '/':
                if second == 0:
                    bot.send_message(message.chat.id, 'На ноль делить нельзя, дурачок!')
                else:
                    res = first / second
                    bot.send_message(message.chat.id, text = str(res))
            bot.register_next_step_handler(message, start_message)
        else:
            bot.send_message(message.chat.id, 'Ты что-то совсем не то ввел, по новой вводи теперь!')
    except:
        bot.send_message(message.chat.id, 'Ой, что-то плохо мне от таких больших чисел, давай попробуем по новой')
               
bot.polling(none_stop=True, interval=0)